In [ ]:
import import_ipynb
import cv2
import numpy as np
import os
import math
import PossibleChar
import detect_chars
import PossiblePlate
import random

In [ ]:
#module level variables
PLATE_WIDTH_PADDING_FACTOR = 1.6
PLATE_HEIGHT_PADDING_FACTOR = 1.9
GAUSSIAN_SMOOTH_FILTER_SIZE = (5, 5)
ADAPTIVE_THRESH_BLOCK_SIZE = 19
ADAPTIVE_THRESH_WEIGHT = 9
SCALAR_BLACK = (0.0, 0.0, 0.0)
SCALAR_WHITE = (255.0, 255.0, 255.0)
SCALAR_YELLOW = (0.0, 255.0, 255.0)
SCALAR_GREEN = (0.0, 255.0, 0.0)
SCALAR_RED = (0.0, 0.0, 255.0)


In [ ]:
def main():
    #Step 1:Opening and Reading the image
    img=cv2.imread("C:/Users/Winni/Desktop/final Mini/LP/car1.jpg")
    if img is None:
        print("image is not loading")
        os.system("pause")
        return
    #end if
    cv2.imshow("car_image",img)
    
    #Step 2: Identifying a list of possible plates
    listOfPossiblePlates=detect_plates(img);
    
    #Step 3:Detecting Characters in the plate
    listOfPossiblePlates = detect_chars.detectCharsInPlates(listOfPossiblePlates)
    
    if len(listOfPossiblePlates) == 0:                          # if no plates were found
        print("\nno license plates were detected\n") 
    else:
        listOfPossiblePlates.sort(key = lambda possiblePlate: len(possiblePlate.strChars), reverse = True)
        licPlate = listOfPossiblePlates[0]
        cv2.imshow("imgPlate", licPlate.imgPlate)           # show crop of plate and threshold of plate
        cv2.imshow("imgThresh", licPlate.imgThresh)
        if len(licPlate.strChars) == 0:
            # if no chars were found in the plate
            print("\nno characters were detected\n\n")  # show message
            return
        drawRedRectangleAroundPlate(img, licPlate)             # draw red rectangle around plate
        print("\nlicense plate read from image = " + licPlate.strChars + "\n")  # write license plate text to std out
        print("----------------------------------------")
    cv2.waitKey(0)  
        return
    def drawRedRectangleAroundPlate(imgOriginalScene, licPlate):

    p2fRectPoints = cv2.boxPoints(licPlate.rrLocationOfPlateInScene)            # get 4 vertices of rotated rect

    cv2.line(imgOriginalScene, tuple(p2fRectPoints[0]), tuple(p2fRectPoints[1]), SCALAR_RED, 2)         # draw 4 red lines
    cv2.line(imgOriginalScene, tuple(p2fRectPoints[1]), tuple(p2fRectPoints[2]), SCALAR_RED, 2)
    cv2.line(imgOriginalScene, tuple(p2fRectPoints[2]), tuple(p2fRectPoints[3]), SCALAR_RED, 2)
    cv2.line(imgOriginalScene, tuple(p2fRectPoints[3]), tuple(p2fRectPoints[0]), SCALAR_RED, 2)
# end function


In [ ]:
def detect_plates(img):
    list_of_possible_plates=[]
    
    #Step1: Preprocessing the image
    height,width,numchannels=img.shape
    gray_img= np.zeros((height,width,1),np.uint8)
    thres_img=np.zeros((height,width,1),np.uint8)
    gray_img,thres_img=preprocess(img)
    cv2.imshow("gray",gray_img)
    cv2.imshow("thresh",thres_img)
    
    #Step2: Find all the contours that used to be chars
    list_possible_chars= findposschars(thres_img)
    
    #Step3 :Draw contour around the possible chars
    imgContours = np.zeros((height, width, 3), np.uint8)
    contours=[]
    for poschar in list_possible_chars:
        contours.append(poschar.contour)
    
    cv2.drawContours(imgContours, contours, -1, SCALAR_WHITE)
    cv2.imshow("Detect PLates: Contours", imgContours)
    
     #Step4 :Group similar characters to be recognized as plates
    List_MatchingChars= detect_chars.findList_MatchingChars(list_possible_chars)
    
    #Step5: Indicating the matching contours in similar colours
    imgContours = np.zeros((height, width, 3), np.uint8)
    for matchChars in List_MatchingChars:
            intRandomBlue = random.randint(0, 255)
            intRandomGreen = random.randint(0, 255)
            intRandomRed = random.randint(0, 255)
            
            contours = []
            for match in matchChars:
                contours.append(match.contour)
            cv2.drawContours(imgContours, contours, -1, (intRandomBlue, intRandomGreen, intRandomRed))
        # end for
    cv2.imshow("MatchingContours_coloured", imgContours)
    
    #Step6: For each group of matching character attempt to extract plate
    for listMc in List_MatchingChars:
        possiblePlate = extractPlate(img, listMc)
        if possiblePlate.imgPlate is not None:                          # if plate was found
            list_of_possible_plates.append(possiblePlate)
    
    for i in range(0, len(list_of_possible_plates)):
            p2fRectPoints = cv2.boxPoints(list_of_possible_plates[i].rrLocationOfPlateInScene)

            cv2.line(imgContours, tuple(p2fRectPoints[0]), tuple(p2fRectPoints[1]), SCALAR_RED, 2)
            cv2.line(imgContours, tuple(p2fRectPoints[1]), tuple(p2fRectPoints[2]), SCALAR_RED, 2)
            cv2.line(imgContours, tuple(p2fRectPoints[2]), tuple(p2fRectPoints[3]), SCALAR_RED, 2)
            cv2.line(imgContours, tuple(p2fRectPoints[3]), tuple(p2fRectPoints[0]), SCALAR_RED, 2)
            
            #print("possible plate " + str(i) + ", click on any image and press a key to continue . . .")

            #cv2.imshow("possiblePlate", list_of_possible_plates[i].imgPlate)
            
    #cv2.waitKey(0)
    return list_of_possible_plates   


In [ ]:
def preprocess(img):
    height,width,numchannels=img.shape
    #converting the image to grayscale
    imgHSV=np.zeros((height,width,3),np.uint8)
    imgHSV=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    imgHue, imgSaturation, imgValue = cv2.split(imgHSV)
    image_grayscale=imgValue
    
    img_contrast_gray=maximizeContrast(image_grayscale)
    #thresholding of image
    height,weight=image_grayscale.shape
    img_blur= np.zeros((height,width,1),np.uint8)
    img_blur= cv2.GaussianBlur(img_contrast_gray, GAUSSIAN_SMOOTH_FILTER_SIZE, 0)
    img_thresh= cv2.adaptiveThreshold(img_blur, 255.0, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, ADAPTIVE_THRESH_BLOCK_SIZE, ADAPTIVE_THRESH_WEIGHT)
    
    return image_grayscale,img_thresh
#endoffunction
    def maximizeContrast(imgGrayscale):

    height, width = imgGrayscale.shape

    imgTopHat = np.zeros((height, width, 1), np.uint8)
    imgBlackHat = np.zeros((height, width, 1), np.uint8)

    structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

    imgTopHat = cv2.morphologyEx(imgGrayscale, cv2.MORPH_TOPHAT, structuringElement)
    imgBlackHat = cv2.morphologyEx(imgGrayscale, cv2.MORPH_BLACKHAT, structuringElement)

    imgGrayscalePlusTopHat = cv2.add(imgGrayscale, imgTopHat)
    imgGrayscalePlusTopHatMinusBlackHat = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

    return imgGrayscalePlusTopHatMinusBlackHat


In [ ]:
def findposschars(thres_img):
    list_possible_chars=[]
    count=0 #count of possible chars
    thres_img_copy=thres_img.copy()
    
    #Step 1: find all contours
    imgContours, contours, Hierarchy = cv2.findContours(thres_img_copy, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    #Step 2: checking if the list of contours each is a possible char or not
    height,width= thres_img.shape
    imgContours = np.zeros((height, width, 3), np.uint8)
    
    for i in range(0,len(contours)):
        cv2.drawContours(imgContours, contours, i, SCALAR_WHITE)
        possibleChar = PossibleChar.PossibleChar(contours[i])
        if detect_chars.checkIfPossibleChar(possibleChar):                   # if contour is a possible char, this does not compare to other chars
            count=count+1                                                   # increment count of possible chars
            list_possible_chars.append(possibleChar) 
    cv2.imshow("Image-contours", imgContours)
    return list_possible_chars


In [ ]:
def extractPlate(imgOriginal,listOfMatchingChars):
    possiblePlate = PossiblePlate.PossiblePlate() 
    listOfMatchingChars.sort(key = lambda matchingChar: matchingChar.intCenterX)
    
    fltPlateCenterX = (listOfMatchingChars[0].intCenterX + listOfMatchingChars[len(listOfMatchingChars) - 1].intCenterX) / 2.0
    fltPlateCenterY = (listOfMatchingChars[0].intCenterY + listOfMatchingChars[len(listOfMatchingChars) - 1].intCenterY) / 2.0
    ptPlateCenter = fltPlateCenterX, fltPlateCenterY
    intPlateWidth = int((listOfMatchingChars[len(listOfMatchingChars) - 1].intBoundingRectX + listOfMatchingChars[len(listOfMatchingChars) - 1].intBoundingRectWidth - listOfMatchingChars[0].intBoundingRectX) * PLATE_WIDTH_PADDING_FACTOR)

    intTotalOfCharHeights = 0
    for matchingChar in listOfMatchingChars:
        intTotalOfCharHeights = intTotalOfCharHeights + matchingChar.intBoundingRectHeight
    
    fltAverageCharHeight = intTotalOfCharHeights / len(listOfMatchingChars)
    intPlateHeight = int(fltAverageCharHeight * PLATE_HEIGHT_PADDING_FACTOR)
    
    fltOpposite = listOfMatchingChars[len(listOfMatchingChars) - 1].intCenterY - listOfMatchingChars[0].intCenterY
    fltHypotenuse = detect_chars.distchars(listOfMatchingChars[0], listOfMatchingChars[len(listOfMatchingChars) - 1])
    fltCorrectionAngleInRad = math.asin(fltOpposite / fltHypotenuse)
    fltCorrectionAngleInDeg = fltCorrectionAngleInRad * (180.0 / math.pi)
    
    possiblePlate.rrLocationOfPlateInScene = ( tuple(ptPlateCenter), (intPlateWidth, intPlateHeight), fltCorrectionAngleInDeg )

    rotationMatrix = cv2.getRotationMatrix2D(tuple(ptPlateCenter), fltCorrectionAngleInDeg, 1.0)
    height, width, numChannels = imgOriginal.shape
    imgRotated = cv2.warpAffine(imgOriginal, rotationMatrix, (width, height))
    imgCropped = cv2.getRectSubPix(imgRotated, (intPlateWidth, intPlateHeight), tuple(ptPlateCenter))
    possiblePlate.imgPlate = imgCropped
    return possiblePlate


In [ ]:
if __name__ == "__main__":
    main()